In [2]:
import time
import pandas as pd
import numpy as np

from functools import reduce

import matplotlib.pyplot as plt
%matplotlib inline

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

In [3]:
## example headless driver

options = Options()
options.headless = True

browser = webdriver.Firefox(executable_path="./drivers/geckodriver", options=options)
browser.get('https://duckduckgo.com')

print('Title: %s' % browser.title)
browser.quit()

Title: DuckDuckGo — Privacy, simplified.


In [56]:
## functions to navigate nba.com/stats pages

def sort_by_name(browser):
    xpath_player_sort = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table/thead/tr/th[2]'
    browser.find_element_by_xpath(xpath_player_sort).click()
    browser.find_element_by_xpath(xpath_player_sort).click()
    return

def select_all_pages(browser):
    xpath_page_selection = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select'
    xpath_all_pages = xpath_page_selection + '/option[1]'
    browser.find_element_by_xpath(xpath_all_pages).click()
    return

def select_per_100(browser):
    xpath_per_mode = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[3]/div/div/label/select'
    browser.find_element_by_xpath(xpath_per_mode).click()
    
    xpath_per_100 = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[3]/div/div/label/select/option[3]'
    browser.find_element_by_xpath(xpath_per_100).click()
    

def select_stat_type(browser, stat_type):
    """
    stat type mappings
    1: Traditional
    2: Advanced
    3: Misc
    4: Scoring
    5: Usage
    6: Opponent
    7: Defense
    """
    
    stat_table_dict = {'traditional': 1, 'advanced': 2, 'misc': 3, 'scoring': 4, 'usage': 5, 'opponent': 6, 'defense':7}
    n = stat_table_dict[stat_type]
    
    ## select the header to get drop down
    xpath_stat_type_button = '/html/body/main/div[2]/div/div[2]/div/nav-dropdown/nav/section[3]/div/a'
    browser.find_element_by_xpath(xpath_stat_type_button).click()

    ## navigate to different stat type
    xpath_stat_type = f'/html/body/main/div[2]/div/div[2]/div/nav-dropdown/nav/section[3]/ul/li[{n}]/a/span'
    browser.find_element_by_xpath(xpath_stat_type).click()
    
    if n in [1, 3, 6, 7]:
        time.sleep(2)
        select_per_100(browser)
    return

def select_season(browser, season):
    seasons_dict = dict(zip(range(2019, 1995, -1), [i for i in range(1, 100)]))
    n = seasons_dict[season]
    ## click 'SEASON' header
    xpath_season_header = '/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select'
    browser.find_element_by_xpath(xpath_season_header).click()
    ## select season
    xpath_select_season = f'/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div/label/select/option[{n}]'
    browser.find_element_by_xpath(xpath_select_season).click()
    return


def remove_equal_cols(df):
    "works with non-unique column names"
    dup_col_idxs = set()
    for i in range(len(df.columns)-1):
        if i in dup_col_idxs: continue
        for j in range(i+1, len(df.columns)):
            if df.iloc[:, i].equals(df.iloc[:, j]):
                dup_col_idxs.add(j)
                
#                 print('equal cols:', df.columns[i], 'and', df.columns[j])
                
    
    ## testing...
    removed_cols = [df.columns[idx] for idx in dup_col_idxs]
#     print(f'REMOVED {len(dup_col_idxs)} Dup Cols\n')
#     print('REMOVED cols:', removed_cols)
    
    df = df.iloc[:, list(set(range(len(df.columns)))-dup_col_idxs)].copy()
    
    return df

In [127]:
## Get df from raw table

def scrape_table(browser, stat_type: str):
    """
    Scrapes stats table present on browser and returns pandas DF. 
    
    1: Traditional
    2: Advanced
    3: Misc
    4: Scoring
    5: Usage
    6: Opponent
    7: Defense
    """
    stat_table_dict = {'traditional': 1, 'advanced': 2, 'misc': 3, 'scoring': 4, 'usage': 5, 'opponent': 6, 'defense':7}
    n = stat_table_dict[stat_type]
    
    ## read in raw table data
    xpath_stats_table = '/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[2]/div[1]'
    raw_table = browser.find_element_by_xpath(xpath_stats_table)
    table = raw_table.text.split('\n')
    
    ## columns for each stat table type...
    if n == 1:  # traditional
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-']
    
    elif n == 2:  # advanced
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'OFFRTG', 'DEFRTG', 'NETRTG', 'AST%', 'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%', 'TO RATIO', 'EFG%', 'TS%', 'USG%', 'PACE', 'PIE']
    
    elif n == 3: # misc
        cols = ['TEAM','AGE','GP','W','L','MIN','PTS OFF TO','2ND PTS','FBPS','PITP','OPP PTS OFF TO','OPP 2ND PTS','OPP FBPS','OPP PITP','BLK','BLKA','PF','PFD']

    elif n == 4: # scoring
        cols = ['TEAM', 'AGE', 'GP', 'W','L','MIN','%FGA 2PT','%FGA 3PT','%PTS 2PT','%PTS 2PT MR','%PTS 3PT','%PTS FBPS','%PTS FT','%PTS OFFTO','%PTS PITP','2FGM %AST','2FGM %UAST','3FGM %AST','3FGM %UAST','FGM %AST','FGM %UAST']
    
    elif n == 5:  # usage, adjusting 'MIN' to 'TOT MIN' as this is what usage provides and is different than MIN per 100 poss.
        cols = ['TEAM', 'AGE', 'GP', 'W', 'L', 'TOT MIN', 'USG%', '%FGM', '%FGA', '%3PM', '%3PA', '%FTM', '%FTA', '%OREB', '%DREB', '%REB', '%AST', '%TOV', '%STL', '%BLK', '%BLKA', '%PF', '%PFD', '%PTS']
    
#     ## MISSING DATA FROM PRIOR 2010'S, not using opponent stats
#     elif table_num == 6: # opponent
#         cols = ['TEAM','GP','W','L','MIN','OPP FGM','OPP FGA','OPP FG%','OPP 3PM','OPP 3PA','OPP 3P%','OPP FTM','OPP FTA','OPP FT%','OPP OREB','OPP DREB','OPP REB','OPP AST','OPP TOV','OPP STL','OPP BLK','OPP BLKA','OPP PF','OPP PFD','OPP PTS','+/-',]
    
    elif n == 7:  # defense
        cols = ['TEAM','AGE','GP','W','L','MIN','DEF RTG','DREB','DREB%','%DREB','STL','STL%','BLK','%BLK','OPP PTS OFF TOV','OPP PTS 2ND CHANCE','OPP PTS FB','OPP PTS PAINT','DEF WS']

    ## extract position of first player row
    for i, line in enumerate(table):
        if line and line[0] == 'A' and len(line.split(' ')) == 2:
            body = table[i:] 
            break
            
    ## parse body of table now and extract player name index, stats
    player_names, all_stats = [], []
    for i in range(0, len(body), 2):
        player = body[i]
        
        if player in player_names:
            ## need to add team name to player index when players have multiple rows
            
#             print('player:', player, 'already processed')
            
            team = body[i+1].split(' ')[0]
            player += ' ' + team
            
#             print('edited player_name:', player)
            
            ## need to add team name to previous same player entry as well as order is different for varying tables within a season
            team = all_stats[-1][0]
            player_names[-1] += ' ' + team
            
#             print('previous player name edited:', player_names[-1])
            
            
        if len(player.split(' ')) > 0 and len(player.split(' ')) < 4:  # avoid weird missing data rows
            player_names.append(player)
            player_stats = body[i+1].split(' ')
            all_stats.append(player_stats)
        
    return pd.DataFrame(all_stats, index=player_names, columns=cols)

In [136]:
def scrape_combine_season_data(season: int, stat_types: list):
    
    ## set up browser
    generic_nba_stats_url = 'https://stats.nba.com/players/traditional/?sort=PTS&dir=-1'
    browser = webdriver.Firefox(executable_path="./drivers/geckodriver")
    browser.get(generic_nba_stats_url)
    time.sleep(4)  # ensure loading...
    
    dfs = []
    
    
#     print('scraping', season)
    
    
    for stat_type in stat_types:
        select_stat_type(browser, stat_type); time.sleep(4)
        select_season(browser, season); time.sleep(4)
        sort_by_name(browser); time.sleep(4)
        select_all_pages(browser); time.sleep(4)
        df = scrape_table(browser, stat_type)
        dfs.append(df)
        time.sleep(4)
        
        
#         print(stat_type, '# cols:', len(df.columns))
     
    
    ## merge all dataframes together on index and only use unique col names
    combined_stats = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)], left_index=True, right_index=True), dfs)
    
    
#     print('\ncombined # cols:', len(combined_stats.columns))
    
    
#     combined_stats = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)
    ## remove the remaining duplicate columns with slightly diff names (i.e. 'DEFRTG' and 'DEF RTG')
    combined_stats = remove_equal_cols(combined_stats)
    
    
#     print('\nsqueezed combined # cols:', len(combined_stats.columns))
    
    

    
    return combined_stats

In [139]:
## TESTING

stat_types = ['traditional', 'advanced', 'misc', 'usage', 'scoring', 'defense']
# stat_types = ['traditional', 'advanced']
stats_05 = scrape_combine_season_data(2005, stat_types)
cols05 = stats_05.columns.sort_values() 
stats_05.head()

KeyboardInterrupt: 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [137]:
## TESTING

stat_types = ['traditional', 'advanced', 'misc', 'usage', 'scoring', 'defense']
# stat_types = ['traditional', 'advanced']
stats_98 = scrape_combine_season_data(1998, stat_types)
stats_98.head()

KeyError: 0

In [134]:
# standard_cols = list(stats_08.columns.sort_values())
# standard_cols == list(stats_98.columns.sort_values())

set(standard_cols) - set(cols98)

{'%STL', 'DEFRTG', 'OPP 2ND PTS', 'OPP FBPS', 'OPP PITP', 'OPP PTS OFF TO'}

In [124]:
## TESTING

stat_types = ['traditional', 'advanced', 'misc', 'usage', 'scoring', 'defense']
# stat_types = ['traditional', 'advanced']
stats_08, dfs08 = scrape_combine_season_data(2008, stat_types)
stats_08.head()

# expecting 89 cols with MIN and TOT MIN cols


scraping 2008
player: Marcus Williams already processed
edited player_name: Marcus Williams GSW
previous player name edited: Marcus Williams SAS
traditional # cols: 28
player: Marcus Williams already processed
edited player_name: Marcus Williams GSW
previous player name edited: Marcus Williams SAS
advanced # cols: 21
player: Marcus Williams already processed
edited player_name: Marcus Williams SAS
previous player name edited: Marcus Williams GSW
misc # cols: 18
player: Marcus Williams already processed
edited player_name: Marcus Williams SAS
previous player name edited: Marcus Williams GSW
usage # cols: 24
player: Marcus Williams already processed
edited player_name: Marcus Williams GSW
previous player name edited: Marcus Williams SAS
scoring # cols: 21
player: Marcus Williams already processed
edited player_name: Marcus Williams GSW
previous player name edited: Marcus Williams SAS
defense # cols: 19

combined # cols: 93
equal cols: DEFRTG and DEF RTG
equal cols: OPP 2ND PTS and OPP PT

,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,%PTS FT,%PTS OFFTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST,DEF WS
Aaron Brooks,HOU,24,80,52,28,52.6,23.5,8.3,20.6,40.4,...,16.7,12.5,32.2,20.2,79.8,66.4,33.6,36.7,63.3,0.186
Aaron Gray,CHI,24,56,26,30,48.9,13.4,5.5,11.4,48.5,...,17.3,12.2,79.6,40.7,59.3,0.0,0.0,40.7,59.3,0.180
Acie Law,ATL,24,55,33,22,51.3,14.9,4.8,12.8,37.4,...,30.2,17.9,40.7,14.0,86.0,77.8,22.2,25.0,75.0,0.261
Adam Morrison,LAL,24,52,23,29,52.4,15.4,5.9,16.4,35.9,...,8.1,21.5,16.3,52.1,47.9,93.8,6.3,68.8,31.3,0.131
Adonal Foyle,ORL,34,10,6,4,48.8,13.4,5.5,8.7,63.6,...,17.6,11.8,82.4,42.9,57.1,0.0,0.0,42.9,57.1,0.297


In [125]:
stats_08.to_string()



"                      TEAM AGE  GP   W   L   MIN   PTS   FGM   FGA   FG%  3PM   3PA   3P%   FTM   FTA   FT%  OREB  DREB   REB   AST   TOV  STL  BLK    PF    FP DD2 TD3    +/- AST RATIO  AST% AST/TO DEFRTG DREB%  EFG% NETRTG OFFRTG OREB%    PACE    PIE  REB% TO RATIO   TS%  USG% 2ND PTS BLKA  FBPS OPP 2ND PTS OPP FBPS OPP PITP OPP PTS OFF TO   PFD  PITP PTS OFF TO  %3PA  %3PM  %AST  %BLK %BLKA %DREB  %FGA  %FGM  %FTA  %FTM %OREB   %PF  %PFD  %PTS  %REB  %STL  %TOV TOT MIN %FGA 2PT %FGA 3PT %PTS 2PT %PTS 2PT MR %PTS 3PT %PTS FBPS %PTS FT %PTS OFFTO %PTS PITP 2FGM %AST 2FGM %UAST 3FGM %AST 3FGM %UAST FGM %AST FGM %UAST  DEF WS\nAaron Brooks           HOU  24  80  52  28  52.6  23.5   8.3  20.6  40.4  3.0   8.1  36.6   3.9   4.5  86.6   0.9   3.3   4.1   6.3   3.3  1.2  0.2   4.0  38.9   2   0    4.5      19.5  20.1   1.90  103.8   6.3  47.6    3.9  107.7   1.8   91.02    8.5   4.1     10.3  52.1  22.6     1.6  1.4   2.8        12.3     11.4     39.2           16.1   4.0   7.6        2.9 

In [63]:
stats_08[['DEFRTG', 'DEF RTG']].describe()
stats_08['DEFRTG'].equals(stats_08['DEF RTG'])

stats_08[stats_08['DEFRTG']!=stats_08['DEF RTG']]

,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,...,%PTS FBPS,%PTS FT,%PTS OFFTO,%PTS PITP,2FGM %AST,2FGM %UAST,3FGM %AST,3FGM %UAST,FGM %AST,FGM %UAST
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,0.0,0.0,0.0,50.0,50.0,50.0,0.0,0.0,50.0,50.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,16.7,16.7,0.0,33.3,100,0.0,100,0.0,100,0.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,0.0,0.0,0.0,50.0,50.0,50.0,0.0,0.0,50.0,50.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,16.7,16.7,0.0,33.3,100,0.0,100,0.0,100,0.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,0.0,0.0,0.0,50.0,50.0,50.0,0.0,0.0,50.0,50.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,16.7,16.7,0.0,33.3,100,0.0,100,0.0,100,0.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,0.0,0.0,0.0,50.0,50.0,50.0,0.0,0.0,50.0,50.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,16.7,16.7,0.0,33.3,100,0.0,100,0.0,100,0.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,0.0,0.0,0.0,50.0,50.0,50.0,0.0,0.0,50.0,50.0
Marcus Williams,SAS,22,2,1,1,48.4,57.1,28.6,28.6,100,...,16.7,16.7,0.0,33.3,100,0.0,100,0.0,100,0.0


In [95]:
## Seeing where in stats08 there are conflicts in these cols that should be equal and are in other years:

# equal cols: DEF RTG and DEFRTG
# equal cols: OPP PTS 2ND CHANCE and OPP 2ND PTS
# equal cols: OPP PTS FB and OPP FBPS
# equal cols: OPP PTS OFF TOV and OPP PTS OFF TO
# equal cols: OPP PTS PAINT and OPP PITP
# equal cols: STL% and %STL

a = "OPP PTS 2ND CHANCE"
b = 'OPP 2ND PTS'

a = 'OPP PTS FB'
b = 'OPP FBPS'

a = 'OPP PTS OFF TOV'
b = 'OPP PTS OFF TO'
    
df = stats_08[ 
#               stats_08['OPP PTS 2ND CHANCE'] != stats_08['OPP 2ND PTS'] #| 
#               stats_08['OPP PTS FB'] != stats_08['OPP FBPS']  
              stats_08['OPP PTS OFF TOV']!= stats_08['OPP PTS OFF TO'] 
#               stats_08['OPP PTS PAINT']!= stats_08['OPP PITP'] 
#               stats_08['STL%']!= stats_08['%STL']
             ]
df.loc[:, [a,b]]

,OPP PTS OFF TOV,OPP PTS OFF TO
Marcus Williams,42.9,16.7
Marcus Williams_2,16.7,42.9


In [126]:
print(stats_08.loc[['Marcus Williams','Marcus Williams_2'], ['TEAM', "OPP PTS 2ND CHANCE", "OPP 2ND PTS", 'OPP PTS FB', 'OPP FBPS']])



KeyError: "None of [Index(['Marcus Williams', 'Marcus Williams_2'], dtype='object')] are in the [index]"

In [25]:
cols98 = list(stats_98.columns.sort_values())
print(len(cols98))
len(set(cols98))

set(cols08) - set(cols98)

87


{'%STL', 'DEFRTG', 'OPP 2ND PTS', 'OPP FBPS', 'OPP PITP', 'OPP PTS OFF TO'}

In [17]:
# print(stats_08['MIN'].head())
stats_08['TOT MIN'].head()

Aaron Brooks     1998.0
Aaron Gray        715.0
Acie Law          560.0
Adam Morrison     711.0
Adonal Foyle       62.0
Name: TOT MIN, dtype: object

In [195]:
def process_write_season_data(df, season, filepath):
    
    df = df.loc[:, standard_cols].copy()
    df.set_index(df.index.astype(str) + ' ' + str(season)[-2:], inplace=True)
    
    with open(filepath, 'w') as f:
        df.to_csv(filepath)
    
    return df    

In [196]:
# a = pd.DataFrame({'a':[1,2,3]})

process_write_season_data(stats_08, 2008, 'TEST-08')
# process_write_season_data(a, 11)

,%3PA,%3PM,%AST,%BLK,%BLKA,%DREB,%FGA,%FGA 2PT,%FGA 3PT,%FGM,...,REB%,STL,TD3,TEAM,TO RATIO,TOT MIN,TOV,TS%,USG%,W
Aaron Brooks 08,35.0,34.0,29.4,4.7,27.0,9.5,24.0,60.5,39.5,21.1,...,4.1,1.2,0,HOU,10.3,1998.0,3.3,52.1,22.6,52
Aaron Gray 08,0.4,0.0,14.7,21.4,26.1,29.8,13.5,99.4,0.6,14.8,...,15.3,1.0,0,CHI,14.3,715.0,2.7,50.8,13.9,26
Acie Law 08,13.8,12.5,43.4,4.8,32.8,14.2,15.7,79.1,20.9,13.5,...,5.1,1.0,0,ATL,10.4,560.0,2.7,49.0,15.7,33
Adam Morrison 08,35.6,31.1,13.8,5.7,11.0,16.1,20.2,57.0,43.0,16.5,...,6.0,0.7,0,LAL,10.5,711.0,2.4,44.8,17.3,23
Adonal Foyle 08,0.0,0.0,4.2,88.9,0.0,35.4,10.1,100,0.0,16.3,...,18.2,0.0,0,ORL,16.7,62.0,2.4,62.3,11.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yao Ming 08,0.1,0.2,12.2,59.3,31.9,30.8,24.3,99.9,0.1,28.5,...,15.2,0.6,0,HOU,14.8,2589.0,4.8,61.8,25.8,50
Yi Jianlian 08,24.9,21.6,10.1,23.7,21.5,30.2,21.4,72.0,28.0,18.2,...,12.2,1.1,0,NJN,10.2,1421.0,2.6,47.4,20.5,26
Zach Randolph 08,13.0,12.1,12.9,10.7,31.6,31.7,28.5,88.9,11.1,29.9,...,14.7,1.2,0,LAC,9.5,1756.0,3.3,53.1,27.7,19
Zaza Pachulia 08,0.5,0.0,9.4,18.7,28.7,29.3,13.3,99.1,0.9,14.8,...,15.3,1.3,0,ATL,15.9,1473.0,3.3,57.1,16.0,45


In [173]:
standard_cols == list(stats_08.columns.drop('MIN').sort_values())

True

In [166]:
len(stats_08.columns.drop)

87

In [158]:
## STARTING COLS

standard_cols = ['%3PA',
                 '%3PM',
                 '%AST',
                 '%BLK',
                 '%BLKA',
                 '%DREB',
                 '%FGA',
                 '%FGA 2PT',
                 '%FGA 3PT',
                 '%FGM',
                 '%FTA',
                 '%FTM',
                 '%OREB',
                 '%PF',
                 '%PFD',
                 '%PTS',
                 '%PTS 2PT',
                 '%PTS 2PT MR',
                 '%PTS 3PT',
                 '%PTS FBPS',
                 '%PTS FT',
                 '%PTS OFFTO',
                 '%PTS PITP',
                 '%REB',
                 '%STL',
                 '%TOV',
                 '+/-',
                 '2FGM %AST',
                 '2FGM %UAST',
                 '2ND PTS',
                 '3FGM %AST',
                 '3FGM %UAST',
                 '3P%',
                 '3PA',
                 '3PM',
                 'AGE',
                 'AST',
                 'AST RATIO',
                 'AST%',
                 'AST/TO',
                 'BLK',
                 'BLKA',
                 'DD2',
                 'DEF WS',
                 'DEFRTG',
                 'DREB',
                 'DREB%',
                 'EFG%',
                 'FBPS',
                 'FG%',
                 'FGA',
                 'FGM',
                 'FGM %AST',
                 'FGM %UAST',
                 'FP',
                 'FT%',
                 'FTA',
                 'FTM',
                 'GP',
                 'L',
                 'NETRTG',
                 'OFFRTG',
                 'OPP 2ND PTS',
                 'OPP FBPS',
                 'OPP PITP',
                 'OPP PTS OFF TO',
                 'OREB',
                 'OREB%',
                 'PACE',
                 'PF',
                 'PFD',
                 'PIE',
                 'PITP',
                 'PTS',
                 'PTS OFF TO',
                 'REB',
                 'REB%',
                 'STL',
                 'TD3',
                 'TEAM',
                 'TO RATIO',
                 'TOT MIN',
                 'TOV',
                 'TS%',
                 'USG%',
                 'W']

In [ ]:
## 